In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
model = YOLO("yolo11n.pt")

path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# Train the model
train_results = model.train(
    data=path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    # workers=100,
    # patience=20,
    imgsz=640,  # training image size
    device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

In [ ]:
import cv2
from matplotlib import pyplot as plt

def run(img):
    results = model(img)
    results[0].show()
    print(results[0])

# Perform object detection on an image
img = cv2.imread("image.jpg")
run(img)
# img = cv2.flip(img, 0)
# run(img)
# img = cv2.flip(img, 1)
# run(img)
# img = cv2.flip(img, 0)
# run(img)